In [ ]:
!pip install tensorflow==2.5
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 454.3 MB 15 kB/s 
     |████████████████████████████████| 4.0 MB 47.0 MB/s 
     |████████████████████████████████| 1.2 MB 34.0 MB/s 
     |████████████████████████████████| 462 kB 47.3 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68718 sha256=b485b2afc5551da81ad2493b44f4d0b2c9514a871f9477c0c248a2caf899a05f
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.13.3
    Uninstalling wrapt-1.13.3:
      Suc

In [ ]:
#Basic libraries
import pandas as pd 
import numpy as np 

#Visualization libraries
import matplotlib.pyplot as plt 
from matplotlib import rcParams
import seaborn as sns
from textblob import TextBlob
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import iplot
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

#NLTK libraries
import nltk
import re
import string
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Machine Learning libraries
import sklearn 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
 

#Metrics libraries
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Miscellanous libraries
from collections import Counter

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Deep learning libraries
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
import pandas as pd
df = pd.read_csv('AbstractDataset_trainSet_FULL.csv')
df.head()

,id,file_name,text,#label
0,1,2111.09381_originalAbstract.txt,"We present MEDCOD, a Medically- Accurate, Emot...",1
1,2,2111.09381_generatedAbstract.txt,"We present MEDCOD, a Medically- Accurate, Emot...",0
2,3,2111.09388_originalAbstract.txt,This work applies Minimum Bayes Risk (MBR) dec...,1
3,4,2111.09388_generatedAbstract.txt,This work applies Minimum Bayes Risk (MBR) dec...,0
4,5,2111.09412_originalAbstract.txt,"In this study, we present a meta-learning mode...",1


In [ ]:
#get the independent features
X = df['text']
#X.head()
#get dependent variables
y = df['#label']
#y.head()

In [ ]:
df.shape

(200, 4)

In [ ]:
#Checking dataset for any missing values and those are dropped
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
df = df.dropna()

In [ ]:
#create a copy of this dataset and also reset its index values.
messages = df.copy()
messages.reset_index(inplace = True)
#messages.head(10)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Stopword removal and Stemming
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
corpus[3]

'work appli minimum bay risk mbr decod optim divers auto mate metric translat qualiti au tomat metric machin translat made tremend progress recent par ticular neural metric fine tune human rate e g bleurt comet perform surfac metric term cor relat human judgement experi ment show model learn approxim human defin represent low perform improv obtain translat differ classic beam search output translat much lower likelihood less favor sur face metric like bleu'

In [ ]:
df.shape

(200, 4)

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
#Setting up vocabulary size
voc_size=20000

#One hot encoding 
onehot_repr=[one_hot(words,voc_size)for words in corpus]

In [ ]:
#Setting sentence length
sent_length=5000

#Padding the sentences
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[    0     0     0 ... 15084 19818  6465]
 [    0     0     0 ... 15084 19818  6465]
 [    0     0     0 ...  5613  4016 17507]
 ...
 [    0     0     0 ... 13191 16161  7685]
 [    0     0     0 ... 14699  7772 15534]
 [    0     0     0 ... 14699  7772 15534]]


In [ ]:
from tensorflow.keras.layers import Input, Dense, Bidirectional, GRU, Embedding, Dropout, LSTM
from tensorflow.keras.layers import concatenate, SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

In [ ]:
import numpy as np 
# Converting the X and y as array
X_final=np.array(embedded_docs)
y_final=np.array(y)

#Check shape of X and y final
X_final.shape,y_final.shape

((200, 5000), (200,))

In [ ]:
from sklearn.model_selection import train_test_split
# Train test split of the X and y final
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
max_len = 5000
max_features = 20000
# Input shape
inp = Input(shape=(max_len,))

# Embedding and LSTM
x = Embedding(max_features, 100)(inp)
x = SpatialDropout1D(0.33)(x)
x = Bidirectional(LSTM(35, return_sequences=True))(x)

# Pooling
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])

# Output layer
output = Dense(1, activation='sigmoid')(conc)

model = Model(inputs=inp, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.load_weights('Weights/gru5.h5')
model.fit(X_train, y_train, epochs=1, batch_size=128, verbose=1)

results = model.predict(X_test, batch_size=128, verbose=1)


1/1 [==============================] - 2s 2s/step


In [ ]:
def convert_to_preds(results):
    """Converts probabilistic results in [0, 1] to
    binary values, 0 and 1."""
    return [1 if r > 0.5 else 0 for r in results]

preds = convert_to_preds(results)

In [ ]:

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

#print('AUC: {}'.format(roc_auc_score(preds, y_test)))
print('Accuracy: {}'.format(accuracy_score(preds, y_test)))
print('Precision: {}'.format(precision_score(preds, y_test, average='macro')))
print('Recall: {}'.format(recall_score(preds, y_test, average='macro')))
print('F1: {}'.format(f1_score(preds, y_test, average='macro')))

Accuracy: 0.4696969696969697
Precision: 0.5
Recall: 0.23484848484848486
F1: 0.3195876288659794
